In [4]:
import Pkg
push!(LOAD_PATH, "../../");
Pkg.activate("mcmc_redistricting", shared=true)

using Revise
using RandomNumbers
using LiftedTreeWalk

  Activating project at `~/.julia/environments/mcmc_redistricting`


In [ ]:
pctGraphPath = joinpath(".", "4x4.json")
precinct_name = "precinct"
county_name = "county"
population_col = "TOTPOP"
num_dists = 4
rng_seed = 454190
pop_dev = 0.0
cycle_walk_frac = 0.01
steps = 100000/cycle_walk_frac
edge_weights= "connections"

nodeData = Set([precinct_name, county_name, population_col])

Set{String} with 3 elements:
  "county"
  "TOTPOP"
  "precinct"

In [6]:
outercorners = Set([(0,1), (2,3), (3,7), (11,15), (14,15), (12,13), (8,12), (0,4)])
outermiddle = Set([(1,2), (7,11), (13,14), (4,8)])
innermiddle = Set([(5,6), (6,10), (9,10), (5,9)])

outercorners = Set([(0,1), (2,3), (3,7), (11,15), (14,15), (12,13), (8,12), (0,4)])
outermiddle = Set([(1,2), (7,11), (13,14), (4,8)])
innermiddle = Set([(5,6), (6,10), (9,10), (5,9)])

function edge_weight_func(
    base_graph::BaseGraph, 
    n1::Int, n2::Int;
    weights::Vector{Float64}=Vector{Float64}(undef, 0),
    edge_sets::Vector{Set{Tuple{Int64, Int64}}}=
        Vector{Set{Tuple{Int64, Int64}}}(undef, 0)
)
    edge = (min(n1-1,n2-1), max(n1-1,n2-1))
    for (w, es) in zip(weights, edge_sets)
        if edge ∈ es
            return w
        end
    end
    return 1
end

edge_weight_func (generic function with 1 method)

In [ ]:
base_graph = BaseGraph(pctGraphPath,  population_col,  
                       inc_node_data=nodeData, edge_weights=edge_weights,
                       edge_perimeter_col=edge_weights);

edge_sets = [outercorners, outermiddle, innermiddle]
weight_func(g, n1, n2) = edge_weight_func(g, n1, n2; weights=[2.0,2.5,3.0], 
                                          edge_sets=edge_sets)
modify_edge_weights!(base_graph, weight_func);
for e in keys(base_graph.edge_attributes)
    base_graph.edge_attributes[e]["unit"] = 1
end
graph = MultiLevelGraph(base_graph, [precinct_name]);

In [ ]:
# grab number of possible linking edge choices for each plan in list

plans = [(1, 1, 2, 2, 1, 1, 2, 2, 3, 3, 4, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 3, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 4, 4, 1, 2, 4, 4),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 4, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 3, 3, 4, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 3, 4, 2, 3, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 4, 2, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 3, 4, 1, 3, 4, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 4, 2, 3),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 4, 4, 2, 3, 3, 4, 4),
(1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 1, 4, 3, 4, 4, 4, 3),
(1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4),
(1, 2, 2, 2, 1, 1, 2, 3, 1, 4, 3, 3, 4, 4, 4, 3)]

orbitsize = [1, 8, 4, 2, 8, 8, 4, 4, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 2, 2, 2]
cut_edges_per_plan = [8, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 
                      12, 12, 12, 12, 12, 12, 12, 12]
sp_trees = [256, 16, 16, 16, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
prob_of_cut_edges_g0 = Dict{Int64, Float64}()
prob_of_cut_edges_g1 = Dict{Int64, Float64}()

node_to_district = Dict{Tuple{Vararg{String}}, Int}()

for (pi, p) in enumerate(plans)
    for ii = 1:length(p)
        node_to_district[graph.id_to_partitions[1][ii]] = p[ii]
    end
    ml_partition = MultiLevelPartition(graph, node_to_district)
    rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
    partition = LinkCutPartition(ml_partition, rng);
    cut_edges_w = get_cut_edge_sum(partition)
    cut_edges_uw = get_cut_edge_sum(partition; column="unit")
    sfs = Int(round(exp(get_log_spanning_forests(partition))))
    orbitsz = orbitsize[pi]
    prob_of_cut_edges_g0[cut_edges_uw] = get(prob_of_cut_edges_g0, cut_edges_uw, 0) + sfs*orbitsz
    prob_of_cut_edges_g1[cut_edges_uw] = get(prob_of_cut_edges_g1, cut_edges_uw, 0) + orbitsz
    @show cut_edges_w, cut_edges_uw, sfs, orbitsz
end

n0inv = 1.0/sum(values(prob_of_cut_edges_g0)) # need to normalize
n1inv = 1.0/sum(values(prob_of_cut_edges_g1)) # need to normalize
for ce in keys(prob_of_cut_edges_g0)
    prob_of_cut_edges_g0[ce] *= n0inv
    prob_of_cut_edges_g1[ce] *= n1inv
end

@show prob_of_cut_edges_g0
@show prob_of_cut_edges_g1

(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (22.0, 8, 20736, 1)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (21.5, 10, 8640, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (22.5, 10, 4320, 4)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (18.0, 10, 42025, 2)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (21.0, 11, 5125, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (20.0, 11, 10800, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (21.0, 11, 5125, 4)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (14.5, 11, 108000, 4)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (22.5, 12, 1500, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (20.5, 12, 4500, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (20.5, 12, 4500, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (20.5, 12, 4500, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (22.0, 12, 1875, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (22.0, 12, 1800, 8)
(cut_edges_w, cut_edges_uw, sfs, orbitsz) = (20.5, 12, 4500, 8)
(cut_edges_w, cut_edges_uw, sfs, orb

Dict{Int64, Float64} with 4 entries:
  11 => 0.564937
  10 => 0.166052
  12 => 0.24881
  8  => 0.020201

In [15]:
# uniform measure
constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591241231)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]
measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 

edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:steps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition; column="unit")
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1")
for (k, v) in edge_cut_counts
    f = v/steps
    f_ex = prob_of_cut_edges_g1[k] 
    err = f_ex-f
    @show k, v, f, f_ex, err
end

# # rng = PCG.PCGStateOneseq(UInt64, rng_seed + 12412)
# edge_cut_counts = Dict{Int64, Int64}()
# for ii = 1:steps
#     run_metropolis_hastings!(partition, proposal, measure, 1, rng);
#     cut_edges = MultiScaleMapSampler.get_cut_edge_weights(partition, "unit")
#     edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
# end

# println("run 2")
# for (k, v) in edge_cut_counts
#     f = v/steps
#     f_ex = prob_of_cut_edges_g1[k] 
#     err = f_ex-f
#     @show k, v, f, f_ex, err
# end

run 1
(k, v, f, f_ex, err) = (11, 4101132, 0.2050566, 0.20512820512820515, 7.160512820514331e-5)
(k, v, f, f_ex, err) = (10, 2474109, 0.12370545, 0.11965811965811968, -0.0040473303418803175)
(k, v, f, f_ex, err) = (12, 13245301, 0.66226505, 0.6666666666666667, 0.004401616666666719)
(k, v, f, f_ex, err) = (8, 179458, 0.0089729, 0.008547008547008548, -0.0004258914529914528)


In [17]:
# spanning forest

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591241231)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]
measure = Measure()

edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:steps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition; column="unit")
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1a")
for (k, v) in edge_cut_counts
    f = v/steps
    f_ex = prob_of_cut_edges_g0[k] 
    err = f_ex-f
    @show k, v, f, f_ex, err
end

# edge_cut_counts = Dict{Int64, Int64}()
# for ii = 1:steps
#     run_metropolis_hastings!(partition, proposal, measure, 1, rng);
#     cut_edges = MultiScaleMapSampler.get_cut_edge_weights(partition, "unit")
#     edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
# end

# println("run 1b")
# for (k, v) in edge_cut_counts
#     f = v/steps
#     f_ex = prob_of_cut_edges_g0[k] 
#     err = f_ex-f
#     @show k, v, f, f_ex, err
# end

# # rng = PCG.PCGStateOneseq(UInt64, rng_seed + 12412)

# # METHOD 2; will probably be slightly slower than method 1
# measure = Measure(0.0)
# push_measure!(measure, get_log_linking_edges, 1.0)

# edge_cut_counts = Dict{Int64, Int64}()
# for ii = 1:steps
#     run_metropolis_hastings!(partition, proposal, measure, 1, rng);
#     cut_edges = MultiScaleMapSampler.get_cut_edge_weights(partition, "unit")
#     edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
# end

# println("run 2a")
# for (k, v) in edge_cut_counts
#     f = v/steps
#     f_ex = prob_of_cut_edges_g0[k] 
#     err = f_ex-f
#     @show k, v, f, f_ex, err
# end

# edge_cut_counts = Dict{Int64, Int64}()
# for ii = 1:steps
#     run_metropolis_hastings!(partition, proposal, measure, 1, rng);
#     cut_edges = MultiScaleMapSampler.get_cut_edge_weights(partition, "unit")
#     edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
# end

# println("run 2b")
# for (k, v) in edge_cut_counts
#     f = v/steps
#     f_ex = prob_of_cut_edges_g0[k] 
#     err = f_ex-f
#     @show k, v, f, f_ex, err
# end

run 1a
(k, v, f, f_ex, err) = (11, 11249732, 0.5624866, 0.5649370765894517, 0.0024504765894517444)
(k, v, f, f_ex, err) = (10, 3203291, 0.16016455, 0.1660519481025557, 0.005887398102555702)
(k, v, f, f_ex, err) = (12, 5143143, 0.25715715, 0.24881001786678042, -0.008347132133219581)
(k, v, f, f_ex, err) = (8, 403834, 0.0201917, 0.020200957441212055, 9.2574412120551e-6)


In [37]:
inner_box = Set{Edge}([Edge(10,11), Edge(6,10), Edge(6,7), Edge(7,11)])
outer_middles = Set{Edge}([Edge(2,3), Edge(5,9), Edge(8,12), Edge(14,15)])
outer_corners = Set{Edge}([Edge(1,2), Edge(1,5), Edge(3,4), Edge(4,8),
                           Edge(9,13), Edge(13,14), Edge(12,16), Edge(15,16)])
weights = [2, 3, 4]
edge_sets = [inner_box, outer_middles, outer_corners]

function edge_weighting(bg::BaseGraph, n1::Int64, n2::Int64, weights::Vector, 
                        edge_sets::Vector)
    @assert length(weights) == length(edge_sets)
    e = Edge(n1, n2)
    for ii = 1:length(weights)
        if e in edge_sets[ii]
            return weights[ii]
        end
    end
    return 1.0
end

edge_weight_func(bg, n1, n2) = edge_weighting(bg, n1, n2, weights, edge_sets)

base_graph2 = deepcopy(base_graph)
modify_edge_weights!(base_graph2, edge_weight_func)
using Graphs, SimpleWeightedGraphs
for e in edges(base_graph2.simple_graph)
    @show src(e), dst(e), weight(e)
end

(src(e), dst(e), weight(e)) = (1, 2, 4.0)
(src(e), dst(e), weight(e)) = (2, 3, 3.0)
(src(e), dst(e), weight(e)) = (3, 4, 4.0)
(src(e), dst(e), weight(e)) = (1, 5, 4.0)
(src(e), dst(e), weight(e)) = (2, 6, 1.0)
(src(e), dst(e), weight(e)) = (5, 6, 1.0)
(src(e), dst(e), weight(e)) = (3, 7, 1.0)
(src(e), dst(e), weight(e)) = (6, 7, 2.0)
(src(e), dst(e), weight(e)) = (4, 8, 4.0)
(src(e), dst(e), weight(e)) = (7, 8, 1.0)
(src(e), dst(e), weight(e)) = (5, 9, 3.0)
(src(e), dst(e), weight(e)) = (6, 10, 2.0)
(src(e), dst(e), weight(e)) = (9, 10, 1.0)
(src(e), dst(e), weight(e)) = (7, 11, 2.0)
(src(e), dst(e), weight(e)) = (10, 11, 2.0)
(src(e), dst(e), weight(e)) = (8, 12, 3.0)
(src(e), dst(e), weight(e)) = (11, 12, 1.0)
(src(e), dst(e), weight(e)) = (9, 13, 4.0)
(src(e), dst(e), weight(e)) = (10, 14, 1.0)
(src(e), dst(e), weight(e)) = (13, 14, 4.0)
(src(e), dst(e), weight(e)) = (11, 15, 1.0)
(src(e), dst(e), weight(e)) = (14, 15, 3.0)
(src(e), dst(e), weight(e)) = (12, 16, 4.0)
(src(e), dst(e)

In [39]:
graph = MultiLevelGraph(base_graph, [precinct_name]);
graph2 = MultiLevelGraph(base_graph2, [precinct_name]);

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
partition = MultiLevelPartition(graph2, constraints, num_dists; rng=rng);

proposal = build_forest_recom2(constraints)
measure = Measure(1.0)

for ii = 1:2
    edge_cut_counts = Dict{Int64, Int64}()
    for ii = 1:steps
        run_metropolis_hastings!(partition, proposal, measure, 1, rng);
        tmp_part = MultiLevelPartition(graph, partition.node_to_district)
        cut_edges = get_cut_edge_count(tmp_part)
        edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
    end

    println("run "*string(ii))
    for (k, v) in edge_cut_counts
        @show k, v, v/steps
    end
end


run 1
(k, v, v / steps) = (11, 20093, 0.20093)
(k, v, v / steps) = (10, 10194, 0.10194)
(k, v, v / steps) = (12, 69005, 0.69005)
(k, v, v / steps) = (8, 708, 0.00708)
run 2
(k, v, v / steps) = (11, 21190, 0.2119)
(k, v, v / steps) = (10, 11132, 0.11132)
(k, v, v / steps) = (12, 66970, 0.6697)
(k, v, v / steps) = (8, 708, 0.00708)


In [40]:
graph = MultiLevelGraph(base_graph, [precinct_name]);
graph2 = MultiLevelGraph(base_graph2, [precinct_name]);

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
partition = MultiLevelPartition(graph2, constraints, num_dists; rng=rng);

proposal = build_forest_recom2(constraints)
measure = Measure(0.0, 1.0)

for ii = 1:2
    edge_cut_counts = Dict{Int64, Int64}()
    for ii = 1:steps
        run_metropolis_hastings!(partition, proposal, measure, 1, rng);
        tmp_part = MultiLevelPartition(graph, partition.node_to_district)
        cut_edges = get_cut_edge_count(tmp_part)
        edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
    end

    println("run "*string(ii))
    for (k, v) in edge_cut_counts
        @show k, v, v/steps
    end
end


run 1
(k, v, v / steps) = (11, 61201, 0.61201)
(k, v, v / steps) = (10, 15915, 0.15915)
(k, v, v / steps) = (12, 17520, 0.1752)
(k, v, v / steps) = (8, 5364, 0.05364)
run 2
(k, v, v / steps) = (11, 60143, 0.60143)
(k, v, v / steps) = (10, 16194, 0.16194)
(k, v, v / steps) = (12, 18504, 0.18504)
(k, v, v / steps) = (8, 5159, 0.05159)


In [45]:
node_to_district = Dict{Tuple{Vararg{String}}, Int}()

normalization = 0
for (pi,p) in enumerate(plans)
    for ii = 1:length(p)
        node_to_district[graph.id_to_partitions[1][ii]] = p[ii]
    end
    partition = MultiLevelPartition(graph2, node_to_district)
    linking_edges = Int(round(exp(get_log_linking_edges(partition))))
    sf = Int(round(exp(get_log_spanning_forests(partition))))
    cut_edges = get_cut_edge_count(partition)
    normalization += sf*orbitsize[pi]
    @show linking_edges, cut_edges, sf, orbitsize[pi]
end
@show 2560000/normalization

(linking_edges, cut_edges, sf, orbitsize[pi]) = (625, 20, 2560000, 1)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2401, 28.0, 20736, 2)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (1000, 30.0, 9216, 2)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2500, 24.0, 665856, 2)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2401, 28.0, 20736, 2)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (1029, 24.0, 147456, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (1764, 24.0, 147456, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (1250, 25.0, 153600, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2500, 24.0, 147456, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2450, 26.0, 117504, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (3456, 26.0, 82944, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (864, 19.0, 3538944, 4)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (2450, 26.0, 55296, 8)
(linking_edges, cut_edges, sf, orbitsize[pi]) = (3000, 25.0, 110592, 8)


0.07367567965814485

In [42]:
+2560000/(2560000+20736+9216+665856+20736+147456+147456+153600+147456+117504+82944+3538944+55296+110592+614400+110592+117504+110592+41472+36864+110592+368640)

0.27561116776451783